In [23]:
import requests
import os
import json
from dotenv import load_dotenv
import pathlib
import pandas as pd

# name the file we want
env_file_name = 'lrhf97_twit_cred.env'

# designate the path of the file
curr_file = pathlib.Path.cwd()

# join them
env_path = curr_file.joinpath(env_file_name)

# load the file
load_dotenv(env_path)

bearer_token = os.getenv("BEARER_TOKEN")


def create_url():
    # Replace with user ID below - RidgeSupply =3413455348
    user_id = 3413455348
    # Right now the max users is hard coded as 10
    max_users = 10
    
    return "https://api.twitter.com/2/users/{}/followers?max_results={}".format(user_id,max_users)

def get_params():
    return {'user.fields': 'id,name,username,created_at,description,public_metrics,verified,location'}

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FollowersLookupPython"
    return r


def connect_to_endpoint(url, params):
    # params['next_token'] = next_token # param object recieved form the create_url function
    response = requests.request("GET", url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def main():
    url = create_url()
    params = get_params()
    json_response = connect_to_endpoint(url, params)
    # print(json.dumps(json_response, indent=4, sort_keys=True))
    return json_response

ridge_followers= main()

200


In [2]:
ridge_followers

{'data': [{'verified': False,
   'username': 'KaplanJonathanE',
   'description': 'Husband+Dad+Writer+Cyclist+Citizen',
   'created_at': '2009-07-27T16:16:11.000Z',
   'name': 'Jonathan Kaplan',
   'public_metrics': {'followers_count': 1013,
    'following_count': 864,
    'tweet_count': 220,
    'listed_count': 26},
   'location': 'Washington, D.C.',
   'id': '60637364'},
  {'verified': False,
   'username': 'kris_hanon',
   'description': '',
   'created_at': '2011-09-14T02:59:38.000Z',
   'name': 'Kris Hanon',
   'public_metrics': {'followers_count': 104,
    'following_count': 468,
    'tweet_count': 518,
    'listed_count': 0},
   'id': '373160054'},
  {'verified': False,
   'username': 'AWalkerMorrison',
   'description': 'Husband, father, cyclist, student @NCSUgeospatial, bike mechanic @rei',
   'created_at': '2022-02-03T18:02:43.000Z',
   'name': 'Andrew Morrison',
   'public_metrics': {'followers_count': 35,
    'following_count': 1446,
    'tweet_count': 43,
    'listed_count

In [13]:
df_follwers=pd.DataFrame.from_dict(ridge_followers['data'])
df_follwers.head(5)

,verified,username,description,created_at,name,public_metrics,location,id
0,False,KaplanJonathanE,Husband+Dad+Writer+Cyclist+Citizen,2009-07-27T16:16:11.000Z,Jonathan Kaplan,"{'followers_count': 1013, 'following_count': 8...","Washington, D.C.",60637364
1,False,kris_hanon,,2011-09-14T02:59:38.000Z,Kris Hanon,"{'followers_count': 104, 'following_count': 46...",NaN,373160054
2,False,AWalkerMorrison,"Husband, father, cyclist, student @NCSUgeospat...",2022-02-03T18:02:43.000Z,Andrew Morrison,"{'followers_count': 35, 'following_count': 144...","Charlotte, USA",1489298184534908930
3,False,Pedalpusher2941,Just a guy nothin fancy - A little bit of this...,2022-05-10T01:35:39.000Z,Kyle,"{'followers_count': 18, 'following_count': 276...",Texas,1523839034723872776
4,False,wykydgt,"Dad, husband, paramedic, adaptive cyclist.",2009-06-23T14:15:03.000Z,Daniel Williams,"{'followers_count': 212, 'following_count': 56...","Kentucky, USA",49985899


In [15]:
df_follwers.description[0]

'Husband+Dad+Writer+Cyclist+Citizen'

## Find likes

In [16]:

def create_liked_url():
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
    tweet_fields = "tweet.fields=text,author_id"
    user_fields = "user.fields=username"
    # Be sure to replace your-user-id with your own user ID or one of an authenticating user
    # You can find a user ID by using the user lookup endpoint
    id = 3413455348
    # You can adjust ids to include a single Tweets.
    # Or you can add to up to 100 comma-separated IDs
    liked_url = "https://api.twitter.com/2/users/{}/liked_tweets".format(id)
    return liked_url, tweet_fields, user_fields

def connect_to_liked_endpoint(liked_url, tweet_fields):
    response = requests.request(
        "GET", liked_url, auth=bearer_oauth, params=tweet_fields)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

def main():
    liked_url, tweet_fields, user_fields = create_liked_url()
    json_response = connect_to_liked_endpoint(liked_url, tweet_fields)
    return json_response

liked_texts =main()
liked_texts

200


{'data': [{'id': '1506734904549249027',
   'edit_history_tweet_ids': ['1506734904549249027'],
   'text': '@sdbeachgirl9 @philgaimon @ridgesupply',
   'author_id': '16949790'},
  {'id': '1506728125706559491',
   'edit_history_tweet_ids': ['1506728125706559491'],
   'text': '@philgaimon Love the socks 🌈 as a proud supporter, a fan of rainbows, and a bonafide cycling sock nut 🤪 Where can I get some? 🤩🤔',
   'author_id': '250376708'},
  {'id': '1506688981005746184',
   'edit_history_tweet_ids': ['1506688981005746184'],
   'text': '@MBTallentire @ridgesupply',
   'author_id': '69469164'},
  {'id': '1506688850239934469',
   'edit_history_tweet_ids': ['1506688850239934469'],
   'text': '@philgaimon where do i get a pair of these? 🌈',
   'author_id': '876379387112849417'},
  {'id': '1506738461939838982',
   'edit_history_tweet_ids': ['1506738461939838982'],
   'text': '@philgaimon If this was a @ridgesupply ad then it totally worked. XL was still in stock!',
   'author_id': '2838151979'},
  {'

In [17]:
df_liked=pd.DataFrame.from_dict(liked_texts['data'])
df_liked.head(5)

,id,edit_history_tweet_ids,text,author_id
0,1506734904549249027,[1506734904549249027],@sdbeachgirl9 @philgaimon @ridgesupply,16949790
1,1506728125706559491,[1506728125706559491],@philgaimon Love the socks 🌈 as a proud suppor...,250376708
2,1506688981005746184,[1506688981005746184],@MBTallentire @ridgesupply,69469164
3,1506688850239934469,[1506688850239934469],@philgaimon where do i get a pair of these? 🌈,876379387112849417
4,1506738461939838982,[1506738461939838982],@philgaimon If this was a @ridgesupply ad then...,2838151979


In [20]:
df_liked['author_id'][0:5]

0              16949790
1             250376708
2              69469164
3    876379387112849417
4            2838151979
Name: author_id, dtype: object

## Retweets

In [18]:

def create_url():
    # User fields are adjustable, options include:
    # created_at, description, entities, id, location, name,
    # pinned_tweet_id, profile_image_url, protected,
    # public_metrics, url, username, verified, and withheld
    user_fields = "user.fields=created_at,description"
    # You can replace the ID given with the Tweet ID you wish to lookup Retweeting users for
    # You can find an ID by using the Tweet lookup endpoint
    id = "1437987598790582277"
    # You can adjust ids to include a single Tweets.
    # Or you can add to up to 100 comma-separated IDs
    url = "https://api.twitter.com/2/tweets/{}/retweeted_by".format(id)
    return url, user_fields


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RetweetedByPython"
    return r

def connect_to_endpoint(url, user_fields):
    response = requests.request("GET", url, auth=bearer_oauth, params=user_fields)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def main():
    url, user_fields = create_url()
    json_response = connect_to_endpoint(url, user_fields)
    return json_response

retweets = main()
retweets



200


{'data': [{'name': '🐟 🇪🇺🇺🇦Allan Paterson, @RebootGB #FBResist',
   'description': "Active Optimist. Humanist Principles: https://t.co/Kbx0pLKDFV… Best ideas on Europe's future: https://t.co/qL7Wi1SSl8 UK: https://t.co/ZmulUQVqCP",
   'id': '540223123',
   'username': 'AllanGPaterson',
   'created_at': '2012-03-29T19:34:36.000Z'},
  {'name': 'Melissa Johnson',
   'description': "No DM's \nSeeker of truth and justice.\n#Resist #BlackLivesMatter #StrongerTogether\n#ResistanceUnited\n#StandWithUkraine",
   'id': '1008473248063991808',
   'username': 'eattherich146',
   'created_at': '2018-06-17T22:15:33.000Z'},
  {'name': 'Gregg Schwinn',
   'description': '',
   'id': '64939212',
   'username': '605erLBM',
   'created_at': '2009-08-12T04:39:34.000Z'},
  {'name': 'Lady Cybele',
   'description': 'PhD, UC Berkeley: School Psychologist \n#Warriors 🏀 Fan, Music Lover, #ProChoice \n#VOTEBLUE in November!   \n(Will chat w/MAGAs but Lies/Insults=BLOCKED) NO DMs!',
   'id': '787141837286875137',


In [19]:
df_retweet=pd.DataFrame. from_dict(retweets['data'])
df_retweet


,name,description,id,username,created_at
0,"🐟 🇪🇺🇺🇦Allan Paterson, @RebootGB #FBResist",Active Optimist. Humanist Principles: https://...,540223123,AllanGPaterson,2012-03-29T19:34:36.000Z
1,Melissa Johnson,No DM's \nSeeker of truth and justice.\n#Resis...,1008473248063991808,eattherich146,2018-06-17T22:15:33.000Z
2,Gregg Schwinn,,64939212,605erLBM,2009-08-12T04:39:34.000Z
3,Lady Cybele,"PhD, UC Berkeley: School Psychologist \n#Warri...",787141837286875137,lady_cybele,2016-10-15T04:03:33.000Z
4,Mctrish,,1115648212231016450,Mctrish5,2019-04-09T16:10:36.000Z
...,...,...,...,...,...
88,KY Educator,,953806657968050176,KYEducator2018,2018-01-18T01:50:02.000Z
89,Sharon Rucker,retired lady who reads way too much (if that i...,2783307971,SharonQwltnut,2014-09-25T02:46:44.000Z
90,T. F. H.,,1279129295550664706,TFH71222518,2020-07-03T19:06:26.000Z
91,🎨📿ArtsyStuffMaker 🇺🇦🌻,"☆ Inspired by Art, Colors & Sleep ☆ #VoteBlue2...",3778668494,ArtsyStuffMaker,2015-10-04T07:23:18.000Z
